In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-04 19:43:02--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-04 19:43:02 (45.4 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(text_fields=['question','text','section'], keyword_fields=['course'])

In [8]:
q='The course has already begun. May I still enroll?'

In [9]:
index.fit(documents)

In [10]:
boost= {'question':3.0, 'section':0.5}
results = index.search(query = q,
                       filter_dict={'course':'data-engineering-zoomcamp'},
                       boost_dict = boost,
                       num_results=5)

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [12]:
from openai import OpenAI

In [13]:
client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":q}]
    
)

In [ ]:
response.choices[0].message.content

In [ ]:
prompt_template="""
You are a course teaching assistant. Please answer the QUESTION based on the CONTEXT.
Use only facts from the CONTEXT when answering the question.
If the CONTEXT does not contain the answer, output NONE.

CONTEXT: {context}
QUESTION: {question}
""".strip()

In [ ]:
context = ""
for doc in results:
    context += f"section: {doc['section']}\n question: {doc['question']}\n\nanswer: {doc['text']}\n\n"

In [ ]:
print(context)

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()

In [ ]:
print(prompt)

### LLM Zoomcamp 1.5 - The RAG Flow Cleaning and Modularizing Code

In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
import minsearch
import json
from openai import OpenAI


In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

index = minsearch.Index(text_fields=['question','text','section'], keyword_fields=['course'])
index.fit(documents)

In [ ]:
def search(query, boost_dict={}, filter_dict={},num_results = 10):
    results = index.search(
                query = query,
                filter_dict=filter_dict,
                boost_dict = boost_dict,
                num_results=num_results
    )
    return results

In [ ]:
def build_prompt(query,search_results):
    prompt_template="""
    You are a course teaching assistant. Please answer the QUESTION based on the CONTEXT.
    Use only facts from the CONTEXT when answering the question.
    If the CONTEXT does not contain the answer, output NONE.
    
    CONTEXT: {context}
    QUESTION: {question}
    """.strip()

    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\n question: {doc['question']}\n\n answer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt
    

In [ ]:
def llm(prompt):
    client = OpenAI()
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
    )
    return response.choices[0].message.content    

In [ ]:
query = 'elasticsearch'

def rag(query):
    boost_dict= {'question':3.0, 'section':0.5}
    #filter_dict={'course':'data-engineering-zoomcamp'}
    filter_dict={}
    num_results=25
    search_results = search(query,boost_dict,filter_dict,num_results)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer
    
answer = rag(query)
print(answer)

### LLM Zoomcamp 1.6 - Search with Elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client = Elasticsearch('htttp://localhost:9200')

In [ ]:
es_client.info()

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
query = "I just discovered the course. Can I still join?"
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [ ]:
search_response = es_client.search(index=index_name, body=search_query)
hits = search_response['hits']['hits']
result_docs = [hit['_source'] for hit in hits]
result_docs

In [ ]:
def elastic_search(query, *args, **kwargs):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    search_response = es_client.search(index=index_name, body=search_query)
    hits = search_response['hits']['hits']
    result_docs = [hit['_source'] for hit in hits]
    return result_docs

In [ ]:
query = "I just found the course. Can I still join it?"
result_docs = elastic_search(query)
result_docs

In [ ]:
def rag(query,search_func):
    search_results = search_func(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer
    
rag(query,elastic_search)

In [ ]:
def rag(query,search_func, build_prompt_func, llm_func):
    search_results = search_func(query)
    prompt = build_prompt_func(query, search_results)
    answer = llm_func(prompt)
    return answer
    
rag(query,elastic_search,build_prompt, llm)

In [ ]:
def rag(query,build_prompt_func, llm_func,search_func, *args, **kwargs):
    search_results = search_func(query, *args, **kwargs)
    prompt = build_prompt_func(query, search_results)
    answer = llm_func(prompt)
    return answer
    
rag(query,build_prompt, llm, elastic_search)